In [1]:
from datetime import datetime
import numpy as np
import tensorflow as tf
from tensorflow import keras

# from IPython.core.interactiveshell import InteractiveShell
# import matplotlib.pyplot as plt
# import seaborn as sns

In [2]:
# 配置项
# # 这个要放到设置中文之前否则还是小方框
# plt.style.use("seaborn")

# # 指定默认字体 用来正常显示中文标签
# plt.rcParams['font.sans-serif'] = ['SimHei']
# # 解决保存图像是负号'-'显示为方块的问题
# plt.rcParams['axes.unicode_minus'] = False

# #全部行都能输出
# InteractiveShell.ast_node_interactivity = "all"

In [7]:
# 假设x是两张图片 每张图片大小是4x4, 3通道, 均值为1, 方差为0.5
x = tf.random.normal([2, 4, 4, 3], mean=1, stddev=0.5)
net = keras.layers.BatchNormalization(axis=-1, # 默认处理最后一维
                                      scale=True,  # 控制缩放 γ
                                      center=True, # 控制偏移 β
                                    )
# 测试模式
out = net(x)
# gamma, beta, moving_mean, moving_variance
# 在测试模式下 这几个参数都不会更新
net.variables

[<tf.Variable 'batch_normalization_1/gamma:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization_1/beta:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_1/moving_mean:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_1/moving_variance:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>]

In [6]:
# 训练模式 只有在训练模式的情况下 在前向传播的过程中, moving_mean, moving_variance才会更新 , 在反向传播的时候gamma, beta才会更新
out = net(x, training=True)
# 前向传播 1 step
net.variables


[<tf.Variable 'batch_normalization/gamma:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization/beta:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization/moving_mean:0' shape=(3,) dtype=float32, numpy=array([0.02623942, 0.02560844, 0.03206938], dtype=float32)>,
 <tf.Variable 'batch_normalization/moving_variance:0' shape=(3,) dtype=float32, numpy=array([0.9749919 , 0.97569054, 0.9777436 ], dtype=float32)>]

In [8]:
for i in range(100):
    out = net(x, training=True)

net.variables

[<tf.Variable 'batch_normalization_1/gamma:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization_1/beta:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_1/moving_mean:0' shape=(3,) dtype=float32, numpy=array([0.625313  , 0.67889136, 0.5486206 ], dtype=float32)>,
 <tf.Variable 'batch_normalization_1/moving_variance:0' shape=(3,) dtype=float32, numpy=array([0.55132335, 0.49576667, 0.49800643], dtype=float32)>]

In [10]:
# 至于损失函数的计算 不必在意 是随意构造的
optimizer = keras.optimizers.SGD(lr=1e-2)
for i in range(10):
    with tf.GradientTape() as tape:
        out = net(x, training=True)
        loss = tf.reduce_mean(tf.pow(out,2)) - 1

    grads = tape.gradient(loss, net.trainable_variables)
    optimizer.apply_gradients(zip(grads, net.trainable_variables))
  
# 训练模式下 只有在反向传播的时候才会更新 
net.variables

W0829 16:41:52.021071 4605359552 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[<tf.Variable 'batch_normalization_1/gamma:0' shape=(3,) dtype=float32, numpy=array([0.93551916, 0.9356133 , 0.93560797], dtype=float32)>,
 <tf.Variable 'batch_normalization_1/beta:0' shape=(3,) dtype=float32, numpy=array([ 3.5949048e-09, -3.9115551e-09, -3.7252899e-11], dtype=float32)>,
 <tf.Variable 'batch_normalization_1/moving_mean:0' shape=(3,) dtype=float32, numpy=array([0.65983444, 0.71637064, 0.5789081 ], dtype=float32)>,
 <tf.Variable 'batch_normalization_1/moving_variance:0' shape=(3,) dtype=float32, numpy=array([0.52655333, 0.4679296 , 0.47029305], dtype=float32)>]